Основными задачами были:
  - Обработка кастомного датасета
  - Fine-tuning модели GPT2
  - Сохранение кастомного токенизатора и модели
  - Генерация текста по введенной строке

В последствие будет создан парсер для создания кастомного датасета,
подобраны веса, количество эпох и прочие настройки под новый датасет.

Сейчас датасет - книга Ведьмак.

In [1]:
!ls

clm_model_save	model_output  tokenizer_output	witcher.txt
eval_tmp.txt	sample_data   train_tmp.txt


In [2]:
!nvidia-smi

Tue Nov 16 19:42:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    91W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-xwn00f83
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-xwn00f83
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-yvxto_s8
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-yvxto_s8


In [4]:
!pip install tensorflow
!pip install huggingface

Data Splitting

In [5]:
from sklearn.model_selection import train_test_split


with open('/content/witcher.txt', 'r') as data:
  dataset = [x for x in data.readlines()]


train, eval = train_test_split(dataset, train_size=.9, random_state=2021)


with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("".join(eval))

In [6]:
import datasets as dt

In [7]:
datasets = dt.load_dataset("text", data_files={
    "train": '/content/train_tmp.txt', 
    "validation": '/content/eval_tmp.txt'
    })

Using custom data configuration default-d4285a15be1288ad


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-d4285a15be1288ad/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datasets["train"][10]

{'text': 'The lance-corporal yanked the reins and turned his horse. '}

Data Preparation

In [9]:
model_checkpoint = "gpt2"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [12]:
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

In [13]:
tokenized_datasets["train"][1]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [10265,
  21497,
  290,
  607,
  1364,
  19341,
  547,
  2042,
  290,
  4171,
  351,
  387,
  368,
  265,
  16911,
  290,
  36214,
  654,
  13,
  327,
  14783,
  427,
  4185,
  1068,
  290,
  220]}

In [14]:
# block_size = tokenizer.model_max_length
block_size = 128

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"hoorn?' 'So you are going to send Cintrian guerrillas to their deaths?' Meve narrowed her eyes. 'In the line of Ciri's emerald eyes turned the shape and size of saucers. to find the girl and force her to marry-' The lance-corporal yanked the reins and turned his horse. Oxenfurt. here. Come here to me, Ciri.' praised me that it was correct. That I am obedient, she told me to write, and that you should not worry. in a wicker basket. Ciri"

Model Preparation

In [18]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
from transformers import create_optimizer, AdamWeightDecay

In [20]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [21]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [22]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
)
validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
)

Model Training

In [23]:
from transformers.keras_callbacks import PushToHubCallback

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-wikitext2"
username = "Rocketknight1"

# a callback to sync up our model with the Hub
# this allows us to resume training from other machines
# and even test the model's inference quality midway through training

'''
callback = PushToHubCallback(
    output_dir="./clm_model_save",
    tokenizer=tokenizer,
    hub_model_id=f"{username}/{push_to_hub_model_id}",
)

model.fit(train_set, validation_data=validation_set, epochs=1, callbacks=[callback])
'''

model.fit(train_set, validation_data=validation_set, epochs=1)

66/66 [==============================] - 112s 1s/step - loss: 4.3748 - val_loss: 3.9868


Evaluation

In [24]:
eval_loss = model.evaluate(validation_set)

8/8 [==============================] - 3s 398ms/step - loss: 3.9868


In [25]:
import math

print(f"Perplexity: {math.exp(eval_loss):.2f}")

Perplexity: 53.88


Saving model and tokenizer

In [26]:
tokenizer.save_pretrained('tokenizer_output')
model.save_pretrained('model_output')

Loading fine-tuned model

In [27]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('/content/tokenizer_output', from_pt=True)
model = TFGPT2LMHeadModel.from_pretrained('/content/model_output')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/model_output.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Generating text

In [28]:
input_ids = tokenizer.encode("The knight moved to your side", return_tensors='tf')

In [29]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=100,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [30]:
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

0: The knight moved to your side, taking a deep breath. 'What am I doing here? 'I'm not talking about you,' she replied simply with an expression of worry on her face - even if it was merely due displeasure at the presence or absence in any way whatsoever.' With such clarity and calmness which most people could only dream out without realizing what had happened behind closed doors under other circumstances... there is no doubt that they did this because these were their wives who received them from Cal

1: The knight moved to your side and approached her like a spear, the other knights were already there. "It is in vain for me!" cried she loudly; 'I'm not at liberty yet! I have no reason whatsoever!' Ciri's mind was cut short by an eye gouge which flared across one of his eyes as he screamed out something loud that sounded far too strong on its own accord.' For some time it had been hidden away beneath everything else around him -'spiteful

2: The knight moved to your side. 'I've got a